# README

Prerequisites

This code needs to be executed on colab, and the dataset used is from docs/role_split_clean.csv in github

The file structure must be available on google drive

```plaintext
└── content/                         
     └── drive/
          └── MyDrive/   
                └── model_fine_tuning/   
                         └── role_split_clean.csv      

```

# prepare training data

In [ ]:
!pip install transformers datasets peft trl accelerate bitsandbytes huggingface_hub -q
!pip install -U bitsandbytes

In [ ]:
!pip install  groq

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
folder_path = '/content/drive/MyDrive/model_fine_tuning/'
import os
files = os.listdir(folder_path)
print(files)

In [ ]:
import pandas as pd
import json
from groq import Groq
import time

client = Groq(api_key="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")

file_path = os.path.join(folder_path, "role_split_clean.csv")
df = pd.read_csv(file_path)
samples = []
for i, row in df.iterrows():
    prompt = f"""You are a professional airline crew member.
Based on the following summary, write a realistic, professional conversation between {row['Party1_Role']} and {row['Party2_Role']}.
Please follow this format:
{row['Party1_Role']}: ...
{row['Party2_Role']}: ...
{row['Party1_Role']}: ...
{row['Party2_Role']}: ...

Keep the conversation professional, natural, and relevant to the situation.
Summary:
{row['objective_summary']}
"""
    try:
        response = client.chat.completions.create(
            model="meta-llama/llama-4-scout-17b-16e-instruct",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=512,
            temperature=0.7
        )

        output = response.choices[0].message.content.strip()

        samples.append({
            "instruction": f"As a {row['Party1_Role']}, engage in a dialogue with {row['Party2_Role']} about the following situation.",
            "input": row["objective_summary"],
            "output": output
        })

        print(f"Sample {i+1} done.")
        time.sleep(0.5)

    except Exception as e:
        print(f" Error at sample {i+1}: {e}")
with open("/content/drive/MyDrive/model_fine_tuning/groq_llama4_train2.jsonl", "w", encoding="utf-8") as f:
    for s in samples:
        f.write(json.dumps(s, ensure_ascii=False) + "\n")

print("All samples saved to groq_llama4_train.jsonl")






In [ ]:
import json


file_path = "/content/drive/MyDrive/model_fine_tuning/groq_llama4_train2.jsonl"


with open(file_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 20:
            break
        data = json.loads(line)
        print(f"\n Sample {i + 1}:")
        print(data.get("output", "[No output field found]"))

In [ ]:
def remove_first_line_if_here(text):
    lines = text.strip().split("\n")
    if lines and lines[0].strip().startswith("Here"):
        return "\n".join(lines[1:]).strip()
    return text.strip()

In [ ]:
import json
import re




# Rule logic:
# Read the role names in the first and second lines of text

# Determine whether both are Flight Attendant

#  If yes, alternate between user and assistant in turn

#  If no, set Flight Attendant to user and the other to assistant

# Return to standard chat format (each sentence has role and content)
def convert_to_chat_format_general(text):

# Matches the form "Speaker: content"
    pattern = r"^([A-Za-z\s()]+):\s*[\"“]?(.*?)[\"”]?$"
    lines = text.strip().splitlines()

    parsed_lines = []
    for line in lines:
        match = re.match(pattern, line.strip())
        if match:
            speaker = match.group(1).strip()
            content = match.group(2).strip()
            parsed_lines.append((speaker, content))


# Less than two lines, unable to determine the role
    if len(parsed_lines) < 2:
        return []

    speaker1, _ = parsed_lines[0]
    speaker2, _ = parsed_lines[1]


# Determine whether there are two Flight Attendants
    both_fa = speaker1.lower().startswith("flight attendant") and speaker2.lower().startswith("flight attendant")

    messages = []
    for i, (speaker, content) in enumerate(parsed_lines):
        if both_fa:
            role = "user" if i % 2 == 0 else "assistant"
        else:
            role = "user" if speaker.lower().startswith("flight attendant") else "assistant"

        messages.append({
            "role": role,
            "content": content
        })

    return messages



def convert_jsonl(input_file, output_file):
    converted = []

    with open(input_file, "r", encoding="utf-8") as infile:
        for line in infile:
            data = json.loads(line)
            output_text = data.get("output", "")
            output_text = remove_first_line_if_here(output_text)



            messages = convert_to_chat_format_general(output_text)



            if messages:
                converted.append({"messages": messages})
            print(messages)
    print(len(converted))

    with open(output_file, "w", encoding="utf-8") as outfile:
        for item in converted:
            outfile.write(json.dumps(item, ensure_ascii=False) + "\n")





input_path =  "/content/drive/MyDrive/model_fine_tuning/groq_llama4_train2.jsonl"
output_path = "/content/drive/MyDrive/model_fine_tuning/converted_chat_messages.jsonl"
convert_jsonl(input_path, output_path)

# fine tung model by using groq_llama4_converted_training dataset

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel
from trl import SFTTrainer
from datasets import load_dataset
from huggingface_hub import notebook_login, HfApi
import torch


In [ ]:
train_dataset = load_dataset("json", data_files="/content/drive/MyDrive/model_fine_tuning/converted_chat_messages.jsonl", split="train", keep_in_memory=True)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()
model_name = "Qwen/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

from transformers import BitsAndBytesConfig
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_enable_fp32_cpu_offload=True  )

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)


In [ ]:

model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["c_attn", "q_proj", "v_proj"]
)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/model_fine_tuning/qwen3-8b-aviation-conflict-ft",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=1,
    push_to_hub=False
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
        args=training_args,

)


trainer.train()

# trainer.push_to_hub(tags=["qwen3", "chat", "aviation-conflict"])
# print("The model has been uploaded to Hugging Face")


In [ ]:
model_name = "Qwen/Qwen3-8B"
base_model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)



In [ ]:
merged_model = PeftModel.from_pretrained(base_model,"/content/drive/MyDrive/model_fine_tuning/qwen3-8b-aviation-conflict-ft/checkpoint-315")
merged_model = merged_model.merge_and_unload()
save_path = "./qwen3-8b-merged-full"
merged_model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
api.create_repo(
    repo_id="Yutao-Zhou/qwen3-8b-aviation-conflict-ft",
    repo_type="model",
    exist_ok=True
)

In [ ]:

api.upload_folder(
    folder_path=save_path,
    path_in_repo="",
    repo_id="Yutao-Zhou/qwen3-8b-aviation-conflict-ft",
    repo_type="model"
)

# test model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name = "Yutao-Zhou/qwen3-8b-aviation-conflict-ft"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)


In [ ]:
dialog_history = []


def multi_turn_chat(user_input, history):
    history.append(f"User: {user_input}")


    prompt = "\n".join(history) + "\nAssistant:"

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
       outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
)

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    reply = full_output.split("Assistant:")[-1].strip()


    for end_marker in ["\nUser:","\nAssistant:","\nAI:","\nSupervisor:"]:
        if end_marker in reply:
            reply = reply.split(end_marker)[0].strip()
            break

    ai_response = reply


    history.append(f"AI: {ai_response}")
    return ai_response, history



while True:
    user_input = input("User：")
    if user_input.lower() in ["exit", "quit"]:
        break
    reply, dialog_history = multi_turn_chat(user_input, dialog_history)
    print("Assistant：" + reply)